# Aggregration of Experiment Results

This notebook pulls results from the `output` folder and aggregates them in a large table that can than be formated, pivoted etc.

In [4]:
from pathlib import Path

OUTPUT_PATH = Path("/work/smt4/thulke/vieweg/SER/Code/sisyphus/output")

## Metrics

In [5]:
import pandas as pd
import glob

dfs = []
for path in map(Path, glob.glob(f"{OUTPUT_PATH}/**/metrics.csv", recursive=True)):
    if "pp" in path.parts:
        continue

    metrics_df = pd.read_csv(path)
    metrics_df["experiment"] = path.parts[-3]
    metrics_df["dataset"] = path.parts[-4]
    dfs.append(metrics_df)

metrics = pd.concat(dfs, axis=0, ignore_index=True)
metrics.head()

,split,metric,value,experiment,dataset
0,train,eer,1.0,lj_finetune,rav
1,val,eer,1.0,lj_finetune,rav
2,test,eer,1.0,lj_finetune,rav
3,train,beer_Happiness,0.0,lj_finetune,rav
4,train,beer_Anger,0.0,lj_finetune,rav


In [11]:
from IPython.display import Markdown, display_html

for split in metrics.split.unique():
    # global metrics
    global_only_metrics = metrics.loc[
        ~metrics.metric.str.contains("_") ^ metrics.metric.str.contains("total")
    ]
    global_only_metrics = global_only_metrics.assign(
        metric=global_only_metrics.metric.apply(lambda s: s.split("_")[0])
    )

    global_pivot_metrics = global_only_metrics.query("split == @split").pivot(
        index="experiment", columns=["dataset", "metric"], values="value"
    )

    # beer per emo
    per_emo_metrics = metrics.loc[
        metrics.metric.str.contains("_") & ~metrics.metric.str.contains("total")
    ]
    per_emo_metrics = per_emo_metrics.assign(
        emotion=per_emo_metrics.metric.apply(lambda s: s.split("_")[-1])
    )

    per_emo_pivot_metrics = per_emo_metrics.query("split == @split").pivot(
        index="experiment", columns=["dataset", "emotion"], values="value"
    )

    # print
    display(Markdown(split))
    display_html(
        global_pivot_metrics.style.set_table_attributes(
            "style='display:inline'"
        )._repr_html_()
        + per_emo_pivot_metrics.style.set_table_attributes(
            "style='display:inline'"
        )._repr_html_(),
        raw=True,
    )

train

dataset 
 rav 
 iem 
 
 
 metric 
 eer 
 beer 
 eer 
 beer 
 
 
 experiment 
   
   
   
   
 
 
 
 
 lj_featureextract 
 1.000000 
 0.000000 
 1.000000 
 0.000000 
 
 
 lj_finetune 
 1.000000 
 0.000000 
 1.000000 
 0.000000 
 
 
 

 
 
 
 dataset 
 rav 
 iem 
 
 
 emotion 
 Happiness 
 Anger 
 Sadness 
 Fear 
 Disgust 
 Surprise 
 Happiness 
 Anger 
 Sadness 
 Neutral 
 
 
 experiment 
   
   
   
   
   
   
   
   
   
   
 
 
 
 
 lj_featureextract 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 lj_finetune 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000

val

dataset 
 rav 
 iem 
 
 
 metric 
 eer 
 beer 
 eer 
 beer 
 
 
 experiment 
   
   
   
   
 
 
 
 
 lj_featureextract 
 1.000000 
 0.000000 
 1.000000 
 0.000000 
 
 
 lj_finetune 
 1.000000 
 0.000000 
 1.000000 
 0.000000 
 
 
 

 
 
 
 dataset 
 rav 
 iem 
 
 
 emotion 
 Happiness 
 Anger 
 Sadness 
 Fear 
 Disgust 
 Surprise 
 Happiness 
 Anger 
 Sadness 
 Neutral 
 
 
 experiment 
   
   
   
   
   
   
   
   
   
   
 
 
 
 
 lj_featureextract 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 lj_finetune 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000

test

dataset 
 rav 
 iem 
 
 
 metric 
 eer 
 beer 
 eer 
 beer 
 
 
 experiment 
   
   
   
   
 
 
 
 
 lj_featureextract 
 1.000000 
 0.000000 
 1.000000 
 0.000000 
 
 
 lj_finetune 
 1.000000 
 0.000000 
 1.000000 
 0.000000 
 
 
 

 
 
 
 dataset 
 rav 
 iem 
 
 
 emotion 
 Happiness 
 Anger 
 Sadness 
 Fear 
 Disgust 
 Surprise 
 Happiness 
 Anger 
 Sadness 
 Neutral 
 
 
 experiment 
   
   
   
   
   
   
   
   
   
   
 
 
 
 
 lj_featureextract 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 lj_finetune 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000